# Harmonic Ratio Explorer v5

## Metabolic Harmony Project - Gut Microbiome Sonification

### Workflow

```
MASTER POOL (800+ ratios, ordered by consonance)
         │
         ▼
┌─────────────────────────────────┐
│  ALLOCATOR                      │
│                                 │
│  1. Define categories + filters │
│  2. For each category:          │
│     - Filter remaining pool     │
│     - Take top N ratios         │
│     - Remove from pool          │
│  3. Export JS/JSON              │
└─────────────────────────────────┘
         │
         ▼
   pathways.js ready!
```

### Mapping Philosophy

| Category | Prime Limit | Polarity | Character |
|----------|-------------|----------|------------|
| **Energy** | 2-5 | Mixed | Foundational, stable |
| **Biosynthesis** | 7-13 | Harmonic (+) | Building, ascending |
| **Degradation** | 7-13 | Subharmonic (-) | Breaking, descending |
| **Salvage** | 2-5 | Subharmonic (-) | Recycling, returning |
| **Other** | 17+ | Mixed | Complex, exotic |

---

## 1. Setup

In [1]:
from math import gcd, log, sqrt, log2, floor
from collections import defaultdict, OrderedDict
import csv
import json

print("✅ Imports loaded")

✅ Imports loaded


In [2]:
# ============================================================
# FOLDING FUNCTIONS (Polarity-Preserving)
# ============================================================

def fold_ratio(n, d, max_harmonic=16, mode="wrap"):
    """
    Fold a ratio into range [1/max_harmonic, max_harmonic].
    
    Preserves polarity:
    - Ratios > 1 stay > 1
    - Ratios < 1 stay < 1
    
    Modes:
    - "wrap" (saw): Hit ceiling → restart from floor
    - "reflect" (tri): Hit ceiling → bounce back
    """
    if n == d:
        return (n, d)
    
    value = n / d
    is_harmonic = value > 1
    
    if not is_harmonic:
        n, d = d, n
        value = n / d
    
    num_bands = int(log2(max_harmonic))
    current_band = floor(log2(value))
    
    if 0 <= current_band < num_bands:
        return (n, d) if is_harmonic else (d, n)
    
    if mode == "wrap":
        target_band = current_band % num_bands
    elif mode == "reflect":
        down = list(range(num_bands - 1, -1, -1))
        up = list(range(1, num_bands - 1))
        pattern = down + up if up else down
        excess = current_band - num_bands
        target_band = pattern[excess % len(pattern)]
    else:
        raise ValueError("mode must be 'wrap' or 'reflect'")
    
    band_shift = current_band - target_band
    new_d = d * (2 ** band_shift)
    g = gcd(n, new_d)
    result_n, result_d = n // g, new_d // g
    
    return (result_n, result_d) if is_harmonic else (result_d, result_n)


print("✅ Folding functions loaded")

✅ Folding functions loaded


In [3]:
# RATIO CLASS
# ============================================================

class Ratio:
    """A musical ratio with harmonic analysis and fold tracking."""
    
    def __init__(self, n, d, original_n=None, original_d=None):
        if d == 0:
            raise ValueError("Denominator cannot be zero")
        g = gcd(n, d) if n > 0 else d
        self.n = n // g if n > 0 else 0
        self.d = d // g
        self.original_n = original_n if original_n is not None else self.n
        self.original_d = original_d if original_d is not None else self.d
    
    def __repr__(self):
        return f"{self.n}/{self.d}"
    
    def __str__(self):
        return f"{self.n}/{self.d}"
    
    def __eq__(self, other):
        if isinstance(other, Ratio):
            return self.n == other.n and self.d == other.d
        return False
    
    def __hash__(self):
        return hash((self.n, self.d))
    
    def value(self):
        return self.n / self.d if self.d != 0 else float('inf')
    
    def cents(self):
        if self.n <= 0:
            return float('-inf')
        return 1200 * log2(self.value())
    
    def consonance(self):
        return self.n * self.d
    
    def original_consonance(self):
        return self.original_n * self.original_d
    
    def original_str(self):
        return f"{self.original_n}/{self.original_d}"
    
    def was_folded(self):
        return self.n != self.original_n or self.d != self.original_d
    
    def _get_prime_factors(self, num):
        if num <= 1:
            return set()
        factors = set()
        d = 2
        temp = num
        while d * d <= temp:
            while temp % d == 0:
                factors.add(d)
                temp //= d
            d += 1
        if temp > 1:
            factors.add(temp)
        return factors
    
    def _get_prime_factorization(self, num):
        if num <= 1:
            return {}
        factors = {}
        d = 2
        temp = num
        while d * d <= temp:
            while temp % d == 0:
                factors[d] = factors.get(d, 0) + 1
                temp //= d
            d += 1
        if temp > 1:
            factors[temp] = factors.get(temp, 0) + 1
        return factors
    
    def prime_factorization(self):
        factors = {}
        for num in [self.n, self.d]:
            for prime, exp in self._get_prime_factorization(num).items():
                factors[prime] = factors.get(prime, 0) + exp
        return factors
    
    def all_primes(self):
        return sorted(self._get_prime_factors(self.n) | self._get_prime_factors(self.d))
    
    def odd_primes(self):
        return set(p for p in self.all_primes() if p > 2)
    
    def odd_prime_exponents(self):
        factors = self.prime_factorization()
        return {p: exp for p, exp in factors.items() if p > 2}
    
    def prime_factors(self):
        all_p = self.all_primes()
        odd_p = [p for p in all_p if p > 2]
        if odd_p:
            return tuple(odd_p)
        elif 2 in all_p:
            return (2,)
        else:
            return (1,)
    
    def prime_factors_str(self):
        pf = self.prime_factors()
        if pf == (1,):
            return "1"
        elif pf == (2,):
            return "2"
        return "×".join(str(p) for p in pf)
    
    def prime_limit(self):
        pf = self.prime_factors()
        return max(pf)
    
    def core(self):
        cn, cd = self.n, self.d
        while cn % 2 == 0:
            cn //= 2
        while cd % 2 == 0:
            cd //= 2
        return (cn, cd)
    
    def _prime_weight(self, x):
        if x <= 1:
            return 0
        total = 0
        temp = x
        d = 2
        while d * d <= temp:
            while temp % d == 0:
                if d > 2:
                    total += log(d)
                temp //= d
            d += 1
        if temp > 2:
            total += log(temp)
        return total
    
    def polarity(self):
        """Harmonic (+1) vs subharmonic (-1) character."""
        cn, cd = self.core()
        n_twos = d_twos = 0
        temp_n, temp_d = self.n, self.d
        while temp_n % 2 == 0:
            temp_n //= 2
            n_twos += 1
        while temp_d % 2 == 0:
            temp_d //= 2
            d_twos += 1
        
        if cn == 1 and cd == 1:
            if self.n > self.d:
                return 1.0
            elif self.n < self.d:
                return -1.0
            return 0.0
        
        if cd == 1:
            return 1.0 / sqrt(1 + d_twos)
        
        if cn == 1:
            return -1.0 / sqrt(1 + n_twos)
        
        n_weight = self._prime_weight(cn)
        d_weight = self._prime_weight(cd)
        if n_weight + d_weight == 0:
            return 0.0
        base = (n_weight - d_weight) / (n_weight + d_weight)
        decay = 1 / sqrt(1 + n_twos + d_twos)
        return base * decay
    
    def is_superparticular(self):
        return self.n == self.d + 1
    
    def is_harmonic(self):
        return self.core()[1] == 1
    
    def is_subharmonic(self):
        return self.core()[0] == 1
    
    def is_odd_squarefree(self):
        for prime, exp in self.odd_prime_exponents().items():
            if exp > 1:
                return False
        return True
    
    def invert(self):
        """Return the subharmonic mirror (d/n)."""
        return Ratio(self.d, self.n, self.original_d, self.original_n)


print("✅ Ratio class loaded")

✅ Ratio class loaded


In [4]:
# ============================================================
# GENERATION FUNCTIONS
# ============================================================

def generate_natural_range(count, min_value, max_value, max_prime_limit=None):
    """Generate exactly `count` ratios that naturally fall within range."""
    result = []
    seen = set()
    consonance = 1
    
    while len(result) < count and consonance < 500000:
        for n in range(1, consonance + 1):
            if consonance % n == 0:
                d = consonance // n
                if gcd(n, d) == 1:
                    value = n / d
                    if not (min_value <= value <= max_value):
                        continue
                    if max_prime_limit:
                        r = Ratio(n, d)
                        if r.prime_limit() > max_prime_limit:
                            continue
                    key = (n, d)
                    if key not in seen:
                        seen.add(key)
                        result.append(Ratio(n, d))
                    if len(result) >= count:
                        break
            if len(result) >= count:
                break
        consonance += 1
    
    return result


def generate_folded(count, max_harmonic=16, fold_mode="wrap", max_prime_limit=None):
    """Generate exactly `count` ratios by consonance, folding into range."""
    result = []
    seen_folded = set()
    consonance = 1
    
    while len(result) < count and consonance < 1000000:
        for n in range(1, consonance + 1):
            if consonance % n == 0:
                d = consonance // n
                if gcd(n, d) == 1:
                    if max_prime_limit:
                        temp_r = Ratio(n, d)
                        if temp_r.prime_limit() > max_prime_limit:
                            continue
                    
                    folded_n, folded_d = fold_ratio(n, d, max_harmonic, fold_mode)
                    folded_key = (folded_n, folded_d)
                    if folded_key in seen_folded:
                        continue
                    
                    seen_folded.add(folded_key)
                    r = Ratio(folded_n, folded_d, original_n=n, original_d=d)
                    result.append(r)
                    
                    if len(result) >= count:
                        break
            if len(result) >= count:
                break
        consonance += 1
    
    return result


print("✅ Generation functions loaded")

✅ Generation functions loaded


In [5]:
# ============================================================
# FILTER FUNCTION
# ============================================================

def filter_ratios(ratio_list, config):
    """
    Filter ratios based on multiple criteria.
    
    Returns: (passed, filtered_out)
    """
    result = ratio_list.copy()
    filtered_out = []
    
    def apply_filter(ratios, condition):
        passed = []
        failed = []
        for r in ratios:
            if condition(r):
                passed.append(r)
            else:
                failed.append(r)
        return passed, failed
    
    # Polarity filters
    if config.get('polarity'):
        pol = config['polarity']
        if pol == 'harmonic':
            result, removed = apply_filter(result, lambda r: r.value() > 1)
        elif pol == 'subharmonic':
            result, removed = apply_filter(result, lambda r: r.value() < 1)
        elif pol == 'both':
            result, removed = apply_filter(result, lambda r: r.value() != 1)
        else:
            removed = []
        filtered_out.extend(removed)
    
    if config.get('polarity_min') is not None:
        result, removed = apply_filter(result, lambda r: r.polarity() >= config['polarity_min'])
        filtered_out.extend(removed)
    
    if config.get('polarity_max') is not None:
        result, removed = apply_filter(result, lambda r: r.polarity() <= config['polarity_max'])
        filtered_out.extend(removed)
    
    # Prime limit filters
    if config.get('prime_limit_max'):
        result, removed = apply_filter(result, lambda r: r.prime_limit() <= config['prime_limit_max'])
        filtered_out.extend(removed)
    
    if config.get('prime_limit_min'):
        result, removed = apply_filter(result, lambda r: r.prime_limit() >= config['prime_limit_min'])
        filtered_out.extend(removed)
    
    if config.get('prime_limit_exact'):
        result, removed = apply_filter(result, lambda r: r.prime_limit() == config['prime_limit_exact'])
        filtered_out.extend(removed)
    
    if config.get('prime_limit_in'):
        result, removed = apply_filter(result, lambda r: r.prime_limit() in config['prime_limit_in'])
        filtered_out.extend(removed)
    
    # Prime factor filters
    if config.get('only_primes'):
        allowed = set(config['only_primes'])
        result, removed = apply_filter(result, lambda r: set(r.all_primes()).issubset(allowed))
        filtered_out.extend(removed)
    
    if config.get('exclude_primes'):
        primes = set(config['exclude_primes'])
        result, removed = apply_filter(result, lambda r: not primes.intersection(set(r.all_primes())))
        filtered_out.extend(removed)
    
    if config.get('has_prime'):
        p = config['has_prime']
        result, removed = apply_filter(result, lambda r: p in r.all_primes())
        filtered_out.extend(removed)
    
    # Factor combination filters
    def has_combo(r, combo):
        if isinstance(combo, str):
            import re
            primes = set(int(p) for p in re.split(r'[x*×]', combo))
        else:
            primes = set(combo)
        return primes.issubset(r.odd_primes())
    
    if config.get('exclude_factor_combos'):
        combos = config['exclude_factor_combos']
        result, removed = apply_filter(result, lambda r: not any(has_combo(r, c) for c in combos))
        filtered_out.extend(removed)
    
    if config.get('has_factor_combo'):
        combo = config['has_factor_combo']
        result, removed = apply_filter(result, lambda r: has_combo(r, combo))
        filtered_out.extend(removed)
    
    # Odd squarefree
    if config.get('odd_squarefree_only'):
        result, removed = apply_filter(result, lambda r: r.is_odd_squarefree())
        filtered_out.extend(removed)
    
    # Consonance filters
    if config.get('consonance_max'):
        result, removed = apply_filter(result, lambda r: r.consonance() <= config['consonance_max'])
        filtered_out.extend(removed)
    
    if config.get('consonance_min'):
        result, removed = apply_filter(result, lambda r: r.consonance() >= config['consonance_min'])
        filtered_out.extend(removed)
    
    # Value range filters
    if config.get('value_min'):
        result, removed = apply_filter(result, lambda r: r.value() >= config['value_min'])
        filtered_out.extend(removed)
    
    if config.get('value_max'):
        result, removed = apply_filter(result, lambda r: r.value() <= config['value_max'])
        filtered_out.extend(removed)
    
    # Include/exclude specific ratios
    if config.get('include_only'):
        include_set = set()
        for item in config['include_only']:
            if isinstance(item, str):
                n, d = map(int, item.split('/'))
            else:
                n, d = item
            include_set.add((n, d))
        result, removed = apply_filter(result, lambda r: (r.n, r.d) in include_set)
        filtered_out.extend(removed)
    
    if config.get('exclude'):
        exclude_set = set()
        for item in config['exclude']:
            if isinstance(item, str):
                n, d = map(int, item.split('/'))
            else:
                n, d = item
            exclude_set.add((n, d))
        result, removed = apply_filter(result, lambda r: (r.n, r.d) not in exclude_set)
        filtered_out.extend(removed)
    
    # Custom filter
    if config.get('custom_filter'):
        result, removed = apply_filter(result, config['custom_filter'])
        filtered_out.extend(removed)
    
    return result, filtered_out


print("✅ Filter function loaded")

✅ Filter function loaded


In [6]:
# ============================================================
# CATEGORY ALLOCATOR
# ============================================================

class CategoryAllocator:
    """
    Allocate ratios from a master pool to categories.
    Tracks used, remaining, and filtered ratios.
    """
    
    def __init__(self, master_pool):
        self.master_pool = list(master_pool)
        self.remaining = list(master_pool)
        self.allocations = OrderedDict()
        self.filtered_out = OrderedDict()
        self.category_filters = OrderedDict()
    
    def define_category(self, name, filter_config, count):
        """Define a category with its filter and how many ratios it needs."""
        self.category_filters[name] = {
            'config': filter_config,
            'count': count
        }
    
    def allocate(self, name, filter_config=None, count=None):
        """
        Allocate ratios to a category.
        
        Args:
            name: Category name
            filter_config: Filter settings (optional if pre-defined)
            count: How many ratios to take (optional if pre-defined)
        
        Returns:
            List of allocated Ratio objects
        """
        if name in self.category_filters:
            if filter_config is None:
                filter_config = self.category_filters[name]['config']
            if count is None:
                count = self.category_filters[name]['count']
        
        if filter_config is None:
            filter_config = {}
        if count is None:
            count = len(self.remaining)
        
        passed, filtered = filter_ratios(self.remaining, filter_config)
        allocated = passed[:count]
        
        allocated_set = set((r.n, r.d) for r in allocated)
        self.remaining = [r for r in self.remaining if (r.n, r.d) not in allocated_set]
        
        self.allocations[name] = allocated
        self.filtered_out[name] = filtered
        
        return allocated
    
    def allocate_all(self):
        """Allocate all pre-defined categories in order."""
        for name in self.category_filters:
            self.allocate(name)
        return self.allocations
    
    def allocate_inverted(self, source_name, target_name, count=None):
        """
        Create inverted (subharmonic mirror) versions of a source category.
        Useful for Degradation = inverted Biosynthesis.
        """
        if source_name not in self.allocations:
            raise ValueError(f"Source '{source_name}' not yet allocated")
        
        source = self.allocations[source_name]
        inverted = [r.invert() for r in source]
        
        if count is not None:
            inverted = inverted[:count]
        
        # Remove from remaining pool
        inverted_set = set((r.n, r.d) for r in inverted)
        self.remaining = [r for r in self.remaining if (r.n, r.d) not in inverted_set]
        
        self.allocations[target_name] = inverted
        return inverted
    
    def summary(self):
        """Print allocation summary."""
        print("\n" + "=" * 70)
        print("ALLOCATION SUMMARY")
        print("=" * 70)
        print(f"Master pool: {len(self.master_pool)} ratios")
        print(f"Remaining:   {len(self.remaining)} ratios")
        print("-" * 70)
        
        total_allocated = 0
        for name, ratios in self.allocations.items():
            needed = self.category_filters.get(name, {}).get('count', '?')
            status = "✓" if len(ratios) >= (needed if isinstance(needed, int) else 0) else "✗"
            print(f"  {name:<35} {len(ratios):>4} / {needed:<4} {status}")
            total_allocated += len(ratios)
        
        print("-" * 70)
        print(f"Total allocated: {total_allocated}")
        print(f"Unallocated:     {len(self.remaining)}")
    
    def get_allocation(self, name):
        return self.allocations.get(name, [])
    
    def get_filtered(self, name):
        return self.filtered_out.get(name, [])
    
    def get_remaining(self):
        return self.remaining
    
    def export_js(self, var_name="RATIO_MAPS"):
        """Export allocations as JavaScript object."""
        lines = [f"const {var_name} = {{"]
        
        for name, ratios in self.allocations.items():
            pairs = ", ".join(f"[{r.n},{r.d}]" for r in ratios)
            safe_name = name.replace("'", "\\'")
            lines.append(f"    '{safe_name}': [{pairs}],")
        
        lines.append("};")
        return "\n".join(lines)
    
    def export_json(self):
        data = {}
        for name, ratios in self.allocations.items():
            data[name] = [[r.n, r.d] for r in ratios]
        return json.dumps(data, indent=2)


print("✅ CategoryAllocator loaded")

✅ CategoryAllocator loaded


In [7]:
# ============================================================
# DISPLAY HELPERS
# ============================================================

def print_ratios(ratios, title="RATIOS", max_display=50):
    """Pretty print ratio list."""
    print(f"\n{'=' * 70}")
    print(f"  {title} ({len(ratios)} ratios)")
    print(f"{'=' * 70}")
    
    if not ratios:
        print("  (empty)")
        return
    
    print(f"  {'#':<4} {'Ratio':<10} {'Value':<10} {'Cents':<10} {'n×d':<8} {'Limit':<6} {'Pol':<8} {'Factors'}")
    print(f"  {'-' * 68}")
    
    for i, r in enumerate(ratios[:max_display], 1):
        print(f"  {i:<4} {str(r):<10} {r.value():<10.4f} {r.cents():>+8.1f}  {r.consonance():<8} {r.prime_limit():<6} {r.polarity():>+6.3f}  {r.prime_factors_str()}")
    
    if len(ratios) > max_display:
        print(f"  ... and {len(ratios) - max_display} more")


def print_as_list(ratios, format='js'):
    """Print complete ratio list in specified format."""
    if format == 'js':
        pairs = [f"[{r.n},{r.d}]" for r in ratios]
        print("[")
        for i in range(0, len(pairs), 10):
            chunk = pairs[i:i+10]
            line = "    " + ", ".join(chunk)
            if i + 10 < len(pairs):
                line += ","
            print(line)
        print("]")
    elif format == 'strings':
        print(" ".join(str(r) for r in ratios))


print("✅ Display helpers loaded")

✅ Display helpers loaded


---

## 2. Pathway Counts

These are the actual counts from your metabolic data.

In [8]:
# ============================================================
# PATHWAY COUNTS (from your data)
# ============================================================

PATHWAY_COUNTS = {
    # ─────────────────────────────────────────────────────────
    # ENERGY (48 total)
    # ─────────────────────────────────────────────────────────
    'Glycolysis/Gluconeogenesis': 11,
    'Pentose Phosphate': 3,
    'Fermentation': 16,
    'TCA Cycle': 14,
    'Glyoxylate Cycle': 4,
    'Respiration': 3,
    # Energy total: 51 (close to 48, some overlap)
    
    # ─────────────────────────────────────────────────────────
    # BIOSYNTHESIS (291 total)
    # ─────────────────────────────────────────────────────────
    'Amino Acids': 50,
    'Nucleotides': 22,
    'Cofactors/Vitamins': 22,
    'Fatty Acids/Lipids': 13,
    'Cell Wall': 6,
    'Polyamines': 6,
    'Biosynthesis Other': 172,
    # Biosynthesis total: 291
    
    # ─────────────────────────────────────────────────────────
    # DEGRADATION (160 total)
    # ─────────────────────────────────────────────────────────
    'Amino Acids Degradation': 21,
    'Nucleotides Degradation': 14,
    'Aromatics Degradation': 13,
    'Carbohydrates Degradation': 9,
    'Degradation Other': 103,
    # Degradation total: 160
    
    # ─────────────────────────────────────────────────────────
    # OTHER CATEGORIES
    # ─────────────────────────────────────────────────────────
    'Salvage': 17,
    'Other': 74,
    'Superpathways': 10,
}

# Calculate totals
energy_total = sum([11, 3, 16, 14, 4, 3])
biosynthesis_total = sum([50, 22, 22, 13, 6, 6, 172])
degradation_total = sum([21, 14, 13, 9, 103])
other_total = sum([17, 74, 10])

print("PATHWAY COUNTS BY CATEGORY")
print("=" * 40)
print(f"  Energy:       {energy_total:>4}")
print(f"  Biosynthesis: {biosynthesis_total:>4}")
print(f"  Degradation:  {degradation_total:>4}")
print(f"  Other:        {other_total:>4}")
print("-" * 40)
print(f"  TOTAL:        {energy_total + biosynthesis_total + degradation_total + other_total:>4}")

PATHWAY COUNTS BY CATEGORY
  Energy:         51
  Biosynthesis:  291
  Degradation:   160
  Other:         101
----------------------------------------
  TOTAL:         603


---

## 3. Generate Master Pool

In [9]:
# ============================================================
# GENERATE MASTER POOL
# ============================================================

# Settings
TOTAL_PATHWAYS = 601
POOL_SIZE = 1200  # Generate more than needed
MIN_VALUE = 1/8
MAX_VALUE = 16
MAX_PRIME_LIMIT = None  # Include some higher primes for "Other" category

# Generate
master = generate_natural_range(
    count=POOL_SIZE,
    min_value=MIN_VALUE,
    max_value=MAX_VALUE,
    max_prime_limit=MAX_PRIME_LIMIT
)

print(f"Generated {len(master)} ratios")
print(f"Range: {MIN_VALUE} to {MAX_VALUE}")
print(f"Prime limit: {MAX_PRIME_LIMIT}")
print(f"Consonance range: {master[0].consonance()} to {master[-1].consonance()}")

Generated 1200 ratios
Range: 0.125 to 16
Prime limit: None
Consonance range: 1 to 806


In [10]:
# Quick look at the pool
print_ratios(master, title="MASTER POOL (first 30)", max_display=30)


  MASTER POOL (first 30) (1200 ratios)
  #    Ratio      Value      Cents      n×d      Limit  Pol      Factors
  --------------------------------------------------------------------
  1    1/1        1.0000         +0.0  1        1      +0.000  1
  2    1/2        0.5000      -1200.0  2        2      -1.000  2
  3    2/1        2.0000      +1200.0  2        2      +1.000  2
  4    1/3        0.3333      -1902.0  3        3      -1.000  3
  5    3/1        3.0000      +1902.0  3        3      +1.000  3
  6    1/4        0.2500      -2400.0  4        2      -1.000  2
  7    4/1        4.0000      +2400.0  4        2      +1.000  2
  8    1/5        0.2000      -2786.3  5        5      -1.000  5
  9    5/1        5.0000      +2786.3  5        5      +1.000  5
  10   1/6        0.1667      -3102.0  6        3      -1.000  3
  11   2/3        0.6667       -702.0  6        3      -0.707  3
  12   3/2        1.5000       +702.0  6        3      +0.707  3
  13   6/1        6.0000      +3102.

---

## 4. Define Category Filters

### Mapping Philosophy

| Category | Prime Limit | Polarity | Rationale |
|----------|-------------|----------|------------|
| **Glycolysis** | 2 only | Harmonic | Pure octaves - most fundamental |
| **Fermentation** | 5 | Subharmonic | Anaerobic - darker, descending |
| **TCA Cycle** | 3 | Harmonic | Pythagorean - clean fifths |
| **Biosynthesis** | 7-13 | Harmonic | Building - ascending |
| **Degradation** | 7-13 | Subharmonic | Breaking - descending (mirror of biosynthesis) |
| **Salvage** | 5 | Subharmonic | Recycling |
| **Other** | 17+ | Mixed | Exotic primes |

In [28]:
# ============================================================
# CATEGORY FILTER DEFINITIONS
# ============================================================

# You can customize these filters!

CATEGORY_FILTERS = {
    # ─────────────────────────────────────────────────────────
    # ENERGY
    # ─────────────────────────────────────────────────────────
    
    'Glycolysis/Gluconeogenesis': {
        'only_primes': [2], # Pure octaves
        # Include 1/1 and subharmonic octaves too
    },
    
    'Pentose Phosphate': {
        'only_primes': [2, 3],       # Pythagorean
        'polarity': 'harmonic',
    },
    
    'Fermentation': {
        'prime_limit_max': 5,
        'exclude_factor_combos': ["3x5"],  # No 15/8, etc.
        'polarity_max': 0,           # Negative polarity only
    },
    
    'TCA Cycle': {
        'only_primes': [2, 3],       # Pythagorean
        'polarity': 'harmonic',
    },
    
    'Glyoxylate Cycle': {
        'only_primes': [2, 3],
        'odd_squarefree_only': True,
    },
    
    'Respiration': {
        'prime_limit_max': 5,
        'polarity': 'harmonic',
    },
    
    # ─────────────────────────────────────────────────────────
    # BIOSYNTHESIS (Prime 7-13, Harmonic)
    # ─────────────────────────────────────────────────────────
    
    'Amino Acids': {
        'prime_limit_in': [5, 7, 11],
        'polarity': 'harmonic',
    },
    
    'Nucleotides': {
        'prime_limit_in': [5, 7, 11],
        'polarity': 'harmonic',
    },
    
    'Cofactors/Vitamins': {
        'prime_limit_in': [7, 11, 13],
        'polarity': 'harmonic',
    },
    
    'Fatty Acids/Lipids': {
        'prime_limit_in': [7, 11, 13],
        'polarity': 'harmonic',
    },
    
    'Cell Wall': {
        'prime_limit_in': [11, 13, 15],
        'polarity': 'harmonic',
    },
    
    'Polyamines': {
        'prime_limit_in': [11, 13, 15],
        'polarity': 'harmonic',
    },
    
    'Biosynthesis Other': {
        'prime_limit_in': [5, 7, 11, 13, 15],
        'polarity': 'harmonic',
    },
    
    # ─────────────────────────────────────────────────────────
    # DEGRADATION (Mirror of Biosynthesis - Subharmonic)
    # Note: We'll use allocate_inverted() for these!
    # ─────────────────────────────────────────────────────────
    
    # These will be created as inversions of biosynthesis
    
    # ─────────────────────────────────────────────────────────
    # SALVAGE
    # ─────────────────────────────────────────────────────────
    
    'Salvage': {
        'prime_limit_max': 5,
        'polarity': 'subharmonic',
    },
    
    # ─────────────────────────────────────────────────────────
    # OTHER (Higher primes)
    # ─────────────────────────────────────────────────────────
    
    'Other': {
        'prime_limit_min': 17,
    },
    
    'Superpathways': {
        # Take whatever's left that's most consonant
    },
}

print("✅ Category filters defined")
print(f"   {len(CATEGORY_FILTERS)} categories")

✅ Category filters defined
   16 categories


---

## 5. Allocate Ratios to Categories

In [29]:
# ============================================================
# CREATE ALLOCATOR
# ============================================================

allocator = CategoryAllocator(master)

print(f"Allocator ready with {len(master)} ratios in pool")

Allocator ready with 1200 ratios in pool


In [30]:
# ============================================================
# ALLOCATE ENERGY CATEGORIES
# ============================================================

# Glycolysis - Pure octaves (including 1/1)
allocator.allocate('Glycolysis/Gluconeogenesis', 
    CATEGORY_FILTERS['Glycolysis/Gluconeogenesis'], 
    PATHWAY_COUNTS['Glycolysis/Gluconeogenesis'])

# Pentose Phosphate
allocator.allocate('Pentose Phosphate',
    CATEGORY_FILTERS['Pentose Phosphate'],
    PATHWAY_COUNTS['Pentose Phosphate'])

# TCA Cycle
allocator.allocate('TCA Cycle',
    CATEGORY_FILTERS['TCA Cycle'],
    PATHWAY_COUNTS['TCA Cycle'])

# Glyoxylate Cycle
allocator.allocate('Glyoxylate Cycle',
    CATEGORY_FILTERS['Glyoxylate Cycle'],
    PATHWAY_COUNTS['Glyoxylate Cycle'])

# Respiration
allocator.allocate('Respiration',
    CATEGORY_FILTERS['Respiration'],
    PATHWAY_COUNTS['Respiration'])

# Fermentation (negative polarity)
allocator.allocate('Fermentation',
    CATEGORY_FILTERS['Fermentation'],
    PATHWAY_COUNTS['Fermentation'])

print("✅ Energy categories allocated")

✅ Energy categories allocated


In [14]:
# ============================================================
# ALLOCATE BIOSYNTHESIS CATEGORIES
# ============================================================

allocator.allocate('Amino Acids',
    CATEGORY_FILTERS['Amino Acids'],
    PATHWAY_COUNTS['Amino Acids'])

allocator.allocate('Nucleotides',
    CATEGORY_FILTERS['Nucleotides'],
    PATHWAY_COUNTS['Nucleotides'])

allocator.allocate('Cofactors/Vitamins',
    CATEGORY_FILTERS['Cofactors/Vitamins'],
    PATHWAY_COUNTS['Cofactors/Vitamins'])

allocator.allocate('Fatty Acids/Lipids',
    CATEGORY_FILTERS['Fatty Acids/Lipids'],
    PATHWAY_COUNTS['Fatty Acids/Lipids'])

allocator.allocate('Cell Wall',
    CATEGORY_FILTERS['Cell Wall'],
    PATHWAY_COUNTS['Cell Wall'])

allocator.allocate('Polyamines',
    CATEGORY_FILTERS['Polyamines'],
    PATHWAY_COUNTS['Polyamines'])

allocator.allocate('Biosynthesis Other',
    CATEGORY_FILTERS['Biosynthesis Other'],
    PATHWAY_COUNTS['Biosynthesis Other'])

print("✅ Biosynthesis categories allocated")

✅ Biosynthesis categories allocated


In [31]:
# ============================================================
# ALLOCATE DEGRADATION AS INVERTED BIOSYNTHESIS
# ============================================================

# The beautiful mirror: Degradation = inverted Biosynthesis
# This creates subharmonic versions of the harmonic biosynthesis ratios

# We'll pull from the combined biosynthesis ratios
all_biosynthesis = []
for cat in ['Amino Acids', 'Nucleotides', 'Cofactors/Vitamins', 
            'Fatty Acids/Lipids', 'Cell Wall', 'Polyamines', 'Biosynthesis Other']:
    all_biosynthesis.extend(allocator.get_allocation(cat))

print(f"Total biosynthesis ratios: {len(all_biosynthesis)}")

# Create inverted versions for degradation
degradation_pool = [r.invert() for r in all_biosynthesis]

# Allocate to degradation categories
deg_idx = 0

# Amino Acids Degradation
count = PATHWAY_COUNTS['Amino Acids Degradation']
allocator.allocations['Amino Acids Degradation'] = degradation_pool[deg_idx:deg_idx+count]
deg_idx += count

# Nucleotides Degradation
count = PATHWAY_COUNTS['Nucleotides Degradation']
allocator.allocations['Nucleotides Degradation'] = degradation_pool[deg_idx:deg_idx+count]
deg_idx += count

# Aromatics Degradation
count = PATHWAY_COUNTS['Aromatics Degradation']
allocator.allocations['Aromatics Degradation'] = degradation_pool[deg_idx:deg_idx+count]
deg_idx += count

# Carbohydrates Degradation
count = PATHWAY_COUNTS['Carbohydrates Degradation']
allocator.allocations['Carbohydrates Degradation'] = degradation_pool[deg_idx:deg_idx+count]
deg_idx += count

# Degradation Other
count = PATHWAY_COUNTS['Degradation Other']
allocator.allocations['Degradation Other'] = degradation_pool[deg_idx:deg_idx+count]
deg_idx += count

print(f"✅ Degradation categories allocated (as inverted biosynthesis)")
print(f"   Used {deg_idx} inverted ratios")

Total biosynthesis ratios: 0
✅ Degradation categories allocated (as inverted biosynthesis)
   Used 160 inverted ratios


In [32]:
# ============================================================
# ALLOCATE REMAINING CATEGORIES
# ============================================================

# Salvage
allocator.allocate('Salvage',
    CATEGORY_FILTERS['Salvage'],
    PATHWAY_COUNTS['Salvage'])

# Other (higher primes)
allocator.allocate('Other',
    CATEGORY_FILTERS['Other'],
    PATHWAY_COUNTS['Other'])

# Superpathways (whatever's left that's most consonant)
allocator.allocate('Superpathways',
    {},  # No filter - just take most consonant remaining
    PATHWAY_COUNTS['Superpathways'])

print("✅ Remaining categories allocated")

✅ Remaining categories allocated


In [33]:
# ============================================================
# SUMMARY
# ============================================================

allocator.summary()


ALLOCATION SUMMARY
Master pool: 1200 ratios
Remaining:   1051 ratios
----------------------------------------------------------------------
  Glycolysis/Gluconeogenesis             8 / ?    ✓
  Pentose Phosphate                      3 / ?    ✓
  TCA Cycle                             14 / ?    ✓
  Glyoxylate Cycle                       4 / ?    ✓
  Respiration                            3 / ?    ✓
  Fermentation                          16 / ?    ✓
  Amino Acids Degradation                0 / ?    ✓
  Nucleotides Degradation                0 / ?    ✓
  Aromatics Degradation                  0 / ?    ✓
  Carbohydrates Degradation              0 / ?    ✓
  Degradation Other                      0 / ?    ✓
  Salvage                               17 / ?    ✓
  Other                                 74 / ?    ✓
  Superpathways                         10 / ?    ✓
----------------------------------------------------------------------
Total allocated: 149
Unallocated:     1051


---

## 6. View Allocations

In [34]:
# View each category
for name in allocator.allocations:
    ratios = allocator.get_allocation(name)
    print_ratios(ratios, title=name, max_display=15)


  Glycolysis/Gluconeogenesis (8 ratios)
  #    Ratio      Value      Cents      n×d      Limit  Pol      Factors
  --------------------------------------------------------------------
  1    1/1        1.0000         +0.0  1        1      +0.000  1
  2    1/2        0.5000      -1200.0  2        2      -1.000  2
  3    2/1        2.0000      +1200.0  2        2      +1.000  2
  4    1/4        0.2500      -2400.0  4        2      -1.000  2
  5    4/1        4.0000      +2400.0  4        2      +1.000  2
  6    1/8        0.1250      -3600.0  8        2      -1.000  2
  7    8/1        8.0000      +3600.0  8        2      +1.000  2
  8    16/1       16.0000     +4800.0  16       2      +1.000  2

  Pentose Phosphate (3 ratios)
  #    Ratio      Value      Cents      n×d      Limit  Pol      Factors
  --------------------------------------------------------------------
  1    3/1        3.0000      +1902.0  3        3      +1.000  3
  2    3/2        1.5000       +702.0  6        3     

In [35]:
# View a specific category in detail
CATEGORY_TO_VIEW = 'Glycolysis/Gluconeogenesis'

ratios = allocator.get_allocation(CATEGORY_TO_VIEW)
print_ratios(ratios, title=CATEGORY_TO_VIEW, max_display=50)

print("\nAs JS array:")
print_as_list(ratios, format='js')


  Glycolysis/Gluconeogenesis (8 ratios)
  #    Ratio      Value      Cents      n×d      Limit  Pol      Factors
  --------------------------------------------------------------------
  1    1/1        1.0000         +0.0  1        1      +0.000  1
  2    1/2        0.5000      -1200.0  2        2      -1.000  2
  3    2/1        2.0000      +1200.0  2        2      +1.000  2
  4    1/4        0.2500      -2400.0  4        2      -1.000  2
  5    4/1        4.0000      +2400.0  4        2      +1.000  2
  6    1/8        0.1250      -3600.0  8        2      -1.000  2
  7    8/1        8.0000      +3600.0  8        2      +1.000  2
  8    16/1       16.0000     +4800.0  16       2      +1.000  2

As JS array:
[
    [1,1], [1,2], [2,1], [1,4], [4,1], [1,8], [8,1], [16,1]
]


---

## 7. Export

In [36]:
# ============================================================
# EXPORT AS JAVASCRIPT
# ============================================================

print(allocator.export_js("SUBCATEGORY_RATIOS"))

const SUBCATEGORY_RATIOS = {
    'Glycolysis/Gluconeogenesis': [[1,1], [1,2], [2,1], [1,4], [4,1], [1,8], [8,1], [16,1]],
    'Pentose Phosphate': [[3,1], [3,2], [6,1]],
    'TCA Cycle': [[9,1], [4,3], [12,1], [9,2], [8,3], [9,4], [16,3], [27,2], [9,8], [32,3], [27,4], [16,9], [27,8], [32,9]],
    'Glyoxylate Cycle': [[1,3], [1,6], [2,3], [3,4]],
    'Respiration': [[5,1], [5,2], [10,1]],
    'Fermentation': [[1,5], [2,5], [2,9], [4,5], [4,9], [8,5], [8,9], [16,5], [4,25], [4,27], [32,5], [8,25], [8,27], [64,5], [16,25], [16,27]],
    'Amino Acids Degradation': [],
    'Nucleotides Degradation': [],
    'Aromatics Degradation': [],
    'Carbohydrates Degradation': [],
    'Degradation Other': [],
    'Salvage': [[3,5], [3,8], [2,15], [3,10], [5,6], [5,8], [5,9], [3,16], [3,20], [4,15], [5,12], [5,16], [5,18], [9,10], [5,24], [8,15], [5,27]],
    'Other': [[17,2], [19,2], [23,2], [3,17], [17,3], [3,19], [19,3], [29,2], [31,2], [4,17], [17,4], [3,23], [23,3], [4,19], [19,4], [5,17], [17,

In [21]:
# ============================================================
# EXPORT TO FILE
# ============================================================

# Save JS
with open('ratio_maps.js', 'w') as f:
    f.write(allocator.export_js("SUBCATEGORY_RATIOS"))
print("✅ Saved to ratio_maps.js")

# Save JSON
with open('ratio_maps.json', 'w') as f:
    f.write(allocator.export_json())
print("✅ Saved to ratio_maps.json")

✅ Saved to ratio_maps.js
✅ Saved to ratio_maps.json


---

## 8. Analysis & Fine-Tuning

In [37]:
# ============================================================
# CHECK FOR GAPS OR ISSUES
# ============================================================

print("CATEGORIES THAT NEED MORE RATIOS:")
print("=" * 50)

for name, ratios in allocator.allocations.items():
    needed = PATHWAY_COUNTS.get(name, 0)
    got = len(ratios)
    if got < needed:
        print(f"  {name}: {got}/{needed} (missing {needed - got})")

CATEGORIES THAT NEED MORE RATIOS:
  Glycolysis/Gluconeogenesis: 8/11 (missing 3)
  Amino Acids Degradation: 0/21 (missing 21)
  Nucleotides Degradation: 0/14 (missing 14)
  Aromatics Degradation: 0/13 (missing 13)
  Carbohydrates Degradation: 0/9 (missing 9)
  Degradation Other: 0/103 (missing 103)


In [39]:
# ============================================================
# VIEW REMAINING (UNALLOCATED) RATIOS
# ============================================================

remaining = allocator.get_remaining()
print_ratios(remaining, title="REMAINING (Unallocated)", max_display=30)


  REMAINING (Unallocated) (1051 ratios)
  #    Ratio      Value      Cents      n×d      Limit  Pol      Factors
  --------------------------------------------------------------------
  1    3/7        0.4286      -1466.9  21       7      -0.278  3×7
  2    7/3        2.3333      +1466.9  21       7      +0.278  3×7
  3    2/11       0.1818      -2951.3  22       11     -0.707  11
  4    11/2       5.5000      +2951.3  22       11     +0.707  11
  5    2/13       0.1538      -3240.5  26       13     -0.707  13
  6    13/2       6.5000      +3240.5  26       13     +0.707  13
  7    4/7        0.5714       -968.8  28       7      -0.577  7
  8    7/4        1.7500       +968.8  28       7      +0.577  7
  9    6/5        1.2000       +315.6  30       5      -0.133  3×5
  10   10/3       3.3333      +2084.4  30       5      +0.133  3×5
  11   15/2       7.5000      +3488.3  30       5      +0.707  3×5
  12   3/11       0.2727      -2249.4  33       11     -0.372  3×11
  13   11/3       

In [24]:
# ============================================================
# FREQUENCY ANALYSIS AT 660 Hz FUNDAMENTAL
# ============================================================

FUNDAMENTAL = 660  # Hz

print(f"\nFREQUENCY RANGE CHECK (fundamental = {FUNDAMENTAL} Hz)")
print("=" * 60)

all_ratios = []
for ratios in allocator.allocations.values():
    all_ratios.extend(ratios)

if all_ratios:
    min_freq = min(FUNDAMENTAL * r.value() for r in all_ratios)
    max_freq = max(FUNDAMENTAL * r.value() for r in all_ratios)
    
    print(f"  Lowest frequency:  {min_freq:.1f} Hz")
    print(f"  Highest frequency: {max_freq:.1f} Hz")
    print(f"  Frequency ratio:   {max_freq/min_freq:.1f}x")
    print(f"  Octaves spanned:   {log2(max_freq/min_freq):.1f}")


FREQUENCY RANGE CHECK (fundamental = 660 Hz)
  Lowest frequency:  41.9 Hz
  Highest frequency: 10560.0 Hz
  Frequency ratio:   252.0x
  Octaves spanned:   8.0


---

## 9. Custom Filtering (Playground)

In [25]:
# ============================================================
# TEST A FILTER CONFIGURATION
# ============================================================

test_config = {
    'prime_limit_max': 5,
    'exclude_factor_combos': ["3x5"],
    'polarity': 'harmonic',
}

passed, filtered = filter_ratios(master, test_config)

print(f"Passed: {len(passed)}")
print(f"Filtered out: {len(filtered)}")
print()
print_ratios(passed, title="TEST FILTER RESULTS", max_display=30)

Passed: 37
Filtered out: 1163


  TEST FILTER RESULTS (37 ratios)
  #    Ratio      Value      Cents      n×d      Limit  Pol      Factors
  --------------------------------------------------------------------
  1    2/1        2.0000      +1200.0  2        2      +1.000  2
  2    3/1        3.0000      +1902.0  3        3      +1.000  3
  3    4/1        4.0000      +2400.0  4        2      +1.000  2
  4    5/1        5.0000      +2786.3  5        5      +1.000  5
  5    3/2        1.5000       +702.0  6        3      +0.707  3
  6    6/1        6.0000      +3102.0  6        3      +1.000  3
  7    8/1        8.0000      +3600.0  8        2      +1.000  2
  8    9/1        9.0000      +3803.9  9        3      +1.000  3
  9    5/2        2.5000      +1586.3  10       5      +0.707  5
  10   10/1       10.0000     +3986.3  10       5      +1.000  5
  11   4/3        1.3333       +498.0  12       3      -0.577  3
  12   12/1       12.0000     +4302.0  12       3      +1.000  3
  13   16/

In [26]:
# ============================================================
# FIND A SPECIFIC RATIO
# ============================================================

def find_ratio(n, d, ratio_list=master):
    target = Ratio(n, d)
    for i, r in enumerate(ratio_list):
        if r == target:
            print(f"Found {r} at index {i+1}:")
            print(f"  Value: {r.value():.6f}")
            print(f"  Cents: {r.cents():+.1f}¢")
            print(f"  Consonance: {r.consonance()}")
            print(f"  Polarity: {r.polarity():+.4f}")
            print(f"  Prime limit: {r.prime_limit()}")
            print(f"  Prime factors: {r.prime_factors_str()}")
            return r
    print(f"{n}/{d} not found")
    return None

# Test
find_ratio(7, 4)

Found 7/4 at index 47:
  Value: 1.750000
  Cents: +968.8¢
  Consonance: 28
  Polarity: +0.5774
  Prime limit: 7
  Prime factors: 7


7/4

In [27]:
# ============================================================
# WHICH CATEGORY IS A RATIO IN?
# ============================================================

def find_category(n, d):
    target = (n, d)
    for name, ratios in allocator.allocations.items():
        for r in ratios:
            if (r.n, r.d) == target:
                print(f"{n}/{d} is in: {name}")
                return name
    print(f"{n}/{d} not allocated to any category")
    return None

# Test
find_category(3, 2)

3/2 is in: Pentose Phosphate


'Pentose Phosphate'